# Shortest Path revisited and NP complete problems

## Bellman-Ford

### Single source shortest path

- input: directed graph $G = (V, E)$, edge lengths $c_{e}$ for each $e \in E$, source vertex $s \in V$
- goal: for every destonation $v \in V$ and source $s$ in $V$, compute the length (sum of edge costs) of the shortest $s-v$ path

### Dijkstra

- good $O(mlogn)$ with heaps
- not always correct with negative edge lenghts, not very distributed

### Negative cycles

- compute the shortest $s-v$ path, with cycles allowed
    - undefined (will keep traversing negative cycle)
- compute shortest cycle-free $s-v$ path
    - NP-hard (no polynomial algorithm, unless P=NP)
- assume input graph has no negative cycle
    - then for every $v$, there is a shortest $s-v$ path with $\le n-1$ edges

### Optimal substructure

- intuition: exploit sequential nature of paths, subpath of a shortest path should itself be shortest
- key idea: artificially restrict the number of edges in path
- lemma: lef $G=(V,E)$ be a directed graph with edge lengths $c_{e}$ and source vertex $s$ ($G$ might or might not have a negative cycle), for every $v \in V, i \in \{1,2 \dots \}$, et $P$ = shortest $s-v$ path with at most $i$ edges (cycles are permitted)
    - case #1: if $P$ has $\le (i-1)$ edges, it is a shortest $s-v$ path with $\le (i-1)$ edges
    - case #2: if $P$ has $i$ edges with last hop $(w,v)$, then $P^{'}$ is a shortest $s-v$ path with $\le (i-1)$ edges
- proof
    - case #1: (obvious) contradiction
    - case #2: if $Q$ (from $s$ to $w$, $\le (i-1)$ edges) is shorter than $P^{'}$, then $Q + (w,v)(=P)$, which contradicts the optimality of $P$

### Recurrence

- let $L_{i,v}$ = minimum length of a s-v path with $\le i$ edges 
    - with cycles allowed
    - defined as $+\infty$ if no $s-v$ paths with $\le i$ edges 
- for every $v \in V$, $i = 1 \dots n$
    - $L_{i,v} = min\left[L_{i-1,v}, min\left[L_{i-1,v},c_{wv}\right]\right]$
- correctness
    - how many candidates are there for an optimal solution to a subproblem involving the destination $v$? $1$+in-degree$(v)$
    - brute-force search from the only ($1$+in-degree$(v)$) candidates (by the optimal substructure lemma)

### If no negative cycles
    - shortest path do not have cycles (removing a cycle only decreases length)
    - have $\le (n-1)$ edges
    - point: if $G$ has no negative cycle, only need to solve subproblems up to $i = n-1$
    - subproblems: compute $L_{i,v}$ for all $i = 1 \dots n-1$ and all $v$ in $V$
    
### Bellman-Ford algorithm

- let $A$ = 2D array (index $i$ and $v$)
- base case: $A[0,s] = 0$ and $A[0,v] = +\infty \ \forall v \ne s$
- for $i = 1 \dots n-1$
    - for each $v \in V$
        - $A[i,v] = min\left[A_{i-1,v}, min\left[A_{i-1,v},c_{wv}\right]\right]$
- if $G$ has no negative cycle, then answer is $A[n-1,v]$
- runs in $O(mn)$

### Stopping early

- suppose for some $j < n-1$, $A[j,v] = A[j-1,v]$ for all vertices $v$
    - for all $v$, all future $A[i,v]'s will be the same$
    - can safely halt (since $A[j-1,v]$'s = correct shortest-path distances)
- claim: $G$ has no negative-cost cycle (that is reachable from $s$) <=> in the extended Bellman-Ford algorithm, $A[n-1,v] = A[n,v] \ \forall v \in V$
- consequence: can check for a negative cycle just by running Bellman-Ford for one extra iteration (run time still $O(mn)$)
- proof
    - (=>) proved in correctness of Bellman-Ford
    - (<=) assume $A[n-1,v] = A[n,v] \ \forall v \in V$, assume also these are finite
    - let $d(v)$ denote the common value of $A[n-1,v]$ and $A[n,v]$
    - notice from algorithm: $d(v) \le d(w) + c_{wv} \ \forall$ edges $(w,v) \in E$
    - $\Sigma_{(w,v) \in c} \ge \Sigma_{(w,v) \in c}(d(w)-d(v)) = 0$

### Predecessor pointers

- only need $A[i-1,v]$'s to compute $A[i,v]$'s
- thus, only need $O(n)$ to remember current and last rounds of subproblems
- compute a second table B where $B[i,v]$ = 2nd-to-last vertex on a shortest s-v path with $\le i $ edges (or NULL if no such path exists)
- reconstruction: assume input graph $G$ has no negative cycles and we correctly compute $B[i,v]$'s
- then, tracing back predecessor pointers $B[n-1,v]$'s from $v$ to $s$ yields a shortest $s-v$ path
- base case: $B[0,v]$ = NULL $\forall v \in V$
- to compute $B[i,v]$ with $i > 0$
    - case 1: $B[i,v] = B[i-1,v]$
    - case 2: $B[i,v]$ = the vertex $x$, $w$ achieving the minimum (ex. the new last hop)
- correctness: computation of $A[i,v]$ is brute-force search through the ($1$+in-degree$(v)$) possible optimal solutions, $B[i,v]$ is just cashing the last hop of the winner
- to reconstruct a negative-cost cycle, use depth-first search to check for a cycle of predessor pointers after each round (must be a negative cost cyle)

### Bellman-Ford to internet routing

- switch from source-drive to destination driven (just reverse all directions in the Bellman-Ford algorithm)
    - every vertex $v$ stores shortest-path distance from $v$ to destination $t$ and the first hop of a shortest-path (for all relevant destinations $t$)
    
### Handling asynchrony

- can't assume all $A[i,v]$'s get computed before all $A[i-1,v]$'s
    - fix: switch from "pull-based" to "push-based": as soon as $A[i,v] \lt A[i-1,v]$, $v$ notifies all of its neighbours
    - algorithm guaranteed to converge eventually (assuming no negative cycles)
        - updates strictly decrease sum of shortest-path estimates
        
### Handling failures

- problem: convergence guaranteed only for static network (not true in practice)
    - fix: each $V$ maintains entire shortest path to $t$, not just the next hop
        - con: more space required
        - pro: more robust to failures, permites more sophisticated route selection

## All pairs shortest path (APSP)

- input: directed graph $G=(V,E)$ with edge cost $c_{e}$ for each edge $e \in E$ (no distinguished source vertex)
- goal: compute the length of a shortest $u->v$ path for all pairs of vertices $u,v \in V$ or correctly that $G$ contains a negative cycle
- Dijkstra runs in $O(nmlogn)$
- Bellman-Ford runs in $O(n^{2}m)$

### Optimal substructure

- key idea: order the vertices $V = {1,2 \dots n}$ arbitrarily, let $V^{k} = {1,2 \dots n}$
- lemma: suppose $G$ has no negative cycle, fix source $i \in V$, destination $j \in V$, and $k \in {1,2 \dots n}$, let $P$ = shortest (cycle-free) $i-j$ path with all internal vertices $V^{(k-1)}$
    - case #1: if $k$ not internal to $P$, then $P$ is a shortest $i-j$ (cycle-free) path with all internal vertices $V^{(k-1)}$
    - case #2: if $k$ is internal to $P$, then
        - $P_{1}$ = shortest (cycle-free) $i-k$ path with all internal nodes in $V^{(k-1)}$
        - $P_{2}$ = shortest (cycle-free) $k-j$ path with all internal nodes in $V^{(k-1)}$

### Floyd-Warshall algorithm
    
- let $A$ = 3D array (index $i,j,k$)
- $A[i,j,k]$ = length of a shortest i-j path with all internal nodes in $\{1 \dots k\}$
- base case: $\forall i,j \in V$
    - $A[i,j,0]$ = 0 if $i$ = $j$, $C_{(ij)}$ if $(i,j)$ in $E$, $+\infty$ if $i$ != $j$ and $(i,j)$ not in $E$
- for $k =1 \dots n$
    - for $i = 1 \dots n$
        - for $j = 1 \dots n$
            $A[i,j,k]$ = $min\left[A[i,j,k-1], A[i,k,k-1] + A[k,j,k-1]\right]$
- runs in $O(n^{3})$ ($O(1)$ per subproblem)
- what if input graph $G$ has a negative cycle? 
    - will have $A[i,i,n] < 0$ for at least one $i \in V$ at the end of algorithm
- reconstruct a shortest $i-j$ path? 
    - in addition to $A$, have Floyd-Warshall compute $B[i,j]$ = max level of an internal node on a shortest i-j path $\forall i,j \in V$ 
    - reset $B[i,j] = k$ if 2nd case of recurrence used to compute $A[i,j,k]$
    - can use $B[i,j]$'s to recursively reconstruct shortest paths!
    
### Rewritting technique

- APSP reduces to $n$ invocations of SSSP
    - non-negative edge length: $O(mnlogn)$ via Dijkstra
    - general edge length: $O(mn^{2})$ via Bellman-Ford
- Johnson's algorithm: reduce all pairs shortest path tp 
    - n Dijkstra ($O(mnlogn)$)
    - 1 Bellman-Ford ($O(mn)$)
- reweighting using vertex weights $\{p_{v}\}$ adds the same amount (namely, $p_{s} - p_{t}$) to every s-t path
- reweighting always leaves the shortest path unchanged

### Reweighting 

- define vertex weight $P_{v}$, which adds the same amount (namely, $P_{s}-P_{t}$ to every $s-t$ path)
- reweighting always leaves the shortest path unchanged
- after reweighting, all edge length are non-negative

### Johnson's algorithm ($O(mnlogn)$)

1. form $G^{'}$ by adding a new vertex $s$ and a new edge $(s,v)$ with length 0 for each $v$ in $G$
2. run Bellman-Ford on $G^{'}$ with source vertex s
3. for each $v$ in $G$, define $p_{v}$ = length of a shortest s-v path in $G^{'}$. For each edge $e=(u,v)$ in $G$, define $c^{'}_{e}$ = $c_{e}$ + $p_{u}$ - $p_{v}$
4. for each vertex $u$ of $G$, run Dijkstra in $G$, with edge lengths $\{c^{'}_{e}\}$, with source vertex $u$, to compute the shortest path distance $d^{'}(u,v)$ for each $v$ in $G$
5. for each pair $u,v$ in $G$, return the shortest path distance $d(u,v)$ = $d^{'}(u,v)$ - $p_{u}$ + $p_{v}$

### Correctness

- claim: for every edge $e=(u,v)$ of $G$, the reweighted length $c^{'}_{e}$ = $c_{e}$ + $p_{u}$ - $p_{v}$ is non-negative
- proof: fix an edge $(u,v)$, by contradiction
     - $p_{u}$ = length of a shortest $s-u$ path of $G^{'}$
     - $p_{v}$ = length of a shortest $s-u$ path of $G^{'}$
     - let $p$ = shortest $s-v$ path in $G^{'}$ (with length $p_{u}$ - exists by constructions of $G^{'}$)
         - $p+(u,v)$ = an $s-v$ path with length $p_{u} + c_{uv}$
         - shortest $s-v$ path only shorter, so $p_{v} \le p_{u} + c_{uv}$
         - $c^{'}_{uv} = c_{uv} + p_{u} - p_{v} \ge 0$

In [ ]:
import numpy as np


def open_file(file_path):
    """
    Read-in a file containing rows of data

    Args:
    file_path (string) -- location of file to read

    Returns:
    tuple_data -- dictionary representing node & weight, integers reprsenting number of vertices and edges
    """

    data_dict = {}

    with open(file_path, 'r') as line:
        data_array = line.read().split("\n")
        num_vertices = int(data_array[0].split(" ")[0])
        num_edges = int(data_array[0].split(" ")[1])
        del data_array[0] # delete first element, which is just metadata
        for item in data_array:
            node1 = item.split(" ")[0]
            node2 = item.split(" ")[1]
            weight = int(item.split(" ")[2])
            data_dict[node1+"-"+node2] = weight
            
    tuple_data = (data_dict, num_vertices, num_edges)
    return tuple_data


def all_pairs_shortest_path(data_dict, num_vertices):
    """
    Implement all pairs shortest path algorithm
    
    Args:
    data_dict (dictionary) -- stores node & weight
    num_vertices (integer) -- number of vertice in graph
    
    Returns:
    smallest (string) -- returns the cost of shortest path (or "Negative cyle" if such exists) 
    """
    
    A = np.zeros((num_vertices, num_vertices, 2))
                  
    for i in range(0, num_vertices):
        for j in range(0, num_vertices):
            index = str(i+1) + "-" + str(j+1)
            if i == j:
                A[i][j][0] = 0
            if index in data_dict:
                A[i][j][0] = data_dict[index]
            if i != j and index not in data_dict:
                A[i][j][0] = 10000000


    smallest = 10000000      
    for k in range(1, num_vertices):
        for i in range(0, num_vertices):
            for j in range(0, num_vertices):
                A[i][j][1] = min(A[i][j][0], A[i][k][0] + A[k][j][0])
                if A[i][j][1] < smallest:
                    smallest = A[i][j][1]
                A[i][j][0] = A[i][j][1]
        print(k)
    
    for i in range(0, num_vertices):
        if A[i][i][1] < 0:
            smallest = "Negative cycle"
    
    if smallest == "Negative cycle":
        return "Negative cycle"
    else:
        return str(int(smallest))

In [ ]:
tuple_obj = open_file("data/all-pairs-shortest-path4.txt")
assert(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]) == "-41")

tuple_obj = open_file("data/all-pairs-shortest-path5.txt")
assert(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]) == "-89")

tuple_obj = open_file("data/all-pairs-shortest-path6.txt")
assert(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]) == "Negative cycle")

tuple_obj = open_file("data/all-pairs-shortest-path7.txt")
assert(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]) == "-2")

tuple_obj = open_file("data/all-pairs-shortest-path1.txt")
assert(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]) == "Negative cycle")

# tuple_obj = open_file("data/all-pairs-shortest-path2.txt")
# print(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]))

# tuple_obj = open_file("data/all-pairs-shortest-path3.txt")
# print(all_pairs_shortest_path(tuple_obj[0], tuple_obj[1]))

# negative
# -19

## NP complete

### Polynomial-time solvability

- there is an algorithm that correctly solves $O(n^{k})$ time
    - $k$ = a constant
    - $n$ = input length
- $P$ = a set of polynomial-time solvable problems
- cycle-free shortest paths in graphs with negative cycles and knapsack are NP-complete problems

### Traveling salesman problem

- input: complete undirected graph with non-negative edge costs
- output: A min-cost tour (cycle that visits every vertex exactly once)
- conjecture: there is no polynomial-time algorithm for TSP

### Reduction

- problem $\sqcap_{1}$ reduces to problem $\sqcap_{2}$ if given a polynomial-time subroutine for $\sqcap_{2}$, can use it to solve $\sqcap_{1}$ in polynomial-time

### Completeness

- suppose $\sqcap_{1}$ reduces to $\sqcap_{2}$
- contrapositive: if $\sqcap_{1}$ is not in $P$, then neither is $\sqcap_{2}$
- that is, $\sqcap_{2}$ is at least as hard as $\sqcap_{1}$
- definition: let $C$ = a set of problems. Problem $\sqcap$ is $C$-complete if
    - $\sqcap \in C$
    - everything in $C$ reduces to $\sqcap$
- that is $\sqcap$ is the hardest problem in all of $C$

### NP-completeness

- a problem is in NP if
    - solutions always have length polynomial in the input size
    - solutions can be verified in polynomial time
- every problem in NP can be solved by brute-force search in exponential time
- vast majority of natural computational problems are in NP
- a polynomial-time algorithm for one NP-complete problem solves every problem in NP efficiently [P = NP]
- but generally, P != NP

### User guide on NP-completeness

- NP-completeness of $\pi$
    - find a known NP-complte problem $\sqcap^{'}$
    - prove that $\sqcap^{'}$ reduced to $\sqcap$
        - implies that $\sqcap$ at least as hard as $\sqcap^{'}$
        - $\sqcap$ is NP-complete as well
                
1. focus on computationally tractable special case
    - WIS in path graphs
    - kanpsack with polynomial size capacity
    - 2 SAT instead of 3 SAT
    - vertex cover when ORT is small
2. heuristics
3. solve in exponential time but faster than brute-force search
    - knapsack ($O(n)$ instead of $2^{n}$)
    - TSP (~$2^{n}$ instead of ~$n!$)
    - vertex cover ($2^{OPT}n$ instead of $n^{OPT}$)

### Vertex cover problem

- given an undirected graph $G=(V,E)$
- compute a minimum-cardinality subset $S \in V$ that contains at least one endpoint of each edge of $G$
- in general, NP-complete problem
- given a positive integer $k$ as input, we want to check whether or not there is a vertex cover with size $\le k$
- could try all possibilities, would take $\theta{n^{k}}$ time

### Substructure lemma

- consider graph $G$, edge $(u,v) \in G$, integer $k \ge 1$
- let $G_{u} = G$ with $u$ and its incident edged deleted
- let $G_{v} = G$ with $v$ and its incident edged deleted
- then, $G$ has a vertex cover of size $k$ <=> $G_{u}$ or $G_{v}$ or both have a vertex cover of size $(k-1)$
- proof
    - (<=) suppose $G_{u}$ (say) has a vertex cover $s$ of size $k-1$, write $E = E_{u}$ (inside $G_{u}$) $\cup$ $F_{u}$ (incident to $u$) since $s$ has an endpoint of each edge of $E_{u}, S \cup \{u\}$ is a vertex cover (of size $k$) of $G$
    - (=>) let $s$ = a vertex cover of $G$ of size $k$, since $(u,v)$ an edge of $G$, at least one of $u,v$ (say $u$) is in $S$, since no edges of $E_{u}$ incident of $u, s-\{u\}$ must be a vertex cover (of size $k-1$) of $G_{u}$

### Recurrence

- ignore base case
- pick an arbitrary edge $(u,v)$ in $E$
- recursively search for a vertex cover $S$ of size $(k-1)$ in $G_{u}$. If found, return $S$ plus $u$
- recursively search for a vertex cover $S$ of size $(k-1)$ in $G_{v}$. If found, return $S$ plus $v$

### Traveling salesman problem

- given undirected graph with non-negative edge cost, find min cost to visit all vertices
- to enforce constraint that each vertex visited exactly once, need to remember the "identities" of vertices visited in a sub-problem
- sub-problem: for every destination $j = \{1 \dots n \}$, every subset $S$ in $\{1 \dots n \}$ that contains $1$ and $j$
    - let $L_{s,j}$ = minimum length of a path from $1$ to $j$ that visits precisely the vertices of $S$ (exactly once each)
- let $p$ be a shortest path from $1$ to $j$ that visits the vertices $S$ (exactly once each) if last hop of $p$ is $(k,j)$, then $p^{'}$ is a shortest path from $1$ to $k$ that visits every vertex of $S - \{j\}$ exactly once

Recurrence
- $L_{s,j} = min\left[L_{s-\{j\},k} + C_{kj}\right]$ ($k$ in $S$, $k != j$)

### Algorithm

- let $A$ = 2D array, indexed by subsets $S$ in $\{1 \dots n\}$ that contain $1$ and destinations $j$ in $\{1 \dots n\}$
- base case: $A[s,1]$ = $0$ if $S = \{1\}$ , $+\infty$ otherwise
- for $m = 2,3,4 \dots n$ [$m$ = sub-problem size]
    - for each set $S \in \{1 \dots n\}$ of size $m$ that contains $1$
        - foe each $j \in S, j=1$
            - $A_{s,j} = min\left[A_{s-\{j\},k} + C_{kj}\right]$ ($k$ in $S$, $k != j$)
- return $min\left[A[\{1,2,3,...,n\}, j] + C_{j1}\right]$
- runs in $O(n2^{n})O(n) = O(n^{2}2^{n})$

In [ ]:
from queue import *
from math import sqrt
import numpy as np


def open_file(file_path):
    """
    Read-in a file containing rows of data

    Args:
    file_path (string) -- location of file to read

    Returns:
    tuple_data (tuple) -- dictionary storing coordinates and integer reprsenting number of cities in total
    """

    data_dict = {}
    index = 0

    with open(file_path, 'r') as line:
        data_array = line.read().split("\n")
        num_cities = int(data_array[0])
        del data_array[0] # delete first element, which is just metadata
        for item in data_array:
            x = float(item.split(" ")[0])
            y = float(item.split(" ")[1])
            data_dict[index] = (x,y)
            index += 1
            
    tuple_data = (data_dict, num_cities)
    return tuple_data


def count_ones(binary):
    """
    Count number of '1' in the binary string
    
    Args:
    binary (string) -- a binary number
    
    Returns:
    count (integer) -- number of '1' in the binary string
    """
    
    count = 0
    for char in binary:
        if char == "1":
            count += 1
            
    return count


def find_ones(m, S):
    """
    Search through binary string in a given list and find binary string whose count of '1's matches the given number  
    
    Args:
    m (integer) -- a number 
    S (list) -- binary strings
    
    Returns
    ret (list) -- binarys string whose count of '1's matches the given number
    """
    
    ret = []
    for binary in S:
        if count_ones(binary) == m:
            ret.append(binary)
            
    return ret


def generate_binary(n, S, max_len):
    """
    Generate binary string
    
    Args:
    n (integer) -- number of binary strings to generate
    S (list) -- stores binary strings
    max_len (integer) -- length of each binary string
    
    Returns:
    None
    """

    # Create an empty queue
    q = Queue()

    # Enqueu the first binary number
    q.put("1")

    # This loop is like BFS of a tree with 1 as root
    # 0 as left child and 1 as right child and so on
    while(n>0):
        n-= 1
        # Print the front of queue
        s1 = q.get()

        zeros = ""
        num_zeros_to_append = max_len - len(s1)
        for i in range(0, num_zeros_to_append):
            zeros += "0"
        S.append(zeros+s1)

        s2 = s1 # Store s1 before changing it

        # Append "0" to s1 and enqueue it
        q.put(s1+"0")

        # Append "1" to s2 and enqueue it. Note that s2
        # contains the previous front
        q.put(s2+"1")


def get_euclidean_distance(a, b, data_dict):
    """
    Find euclidean distance between two 2-dimensional coordinate
    
    Args:
    a (integer) - anindex to locate tuple representing the first coordinate
    b (integer) - an index to locate tuple representing the second coordinate
    data_dict (dictionary) -- stores coordinates
    
    Returns:
    dist (integer) -- euclidean distance between two coordinates
    """
    
    x1 = data_dict[a][0]
    y1 = data_dict[a][1]
    x2 = data_dict[b][0]
    y2 = data_dict[b][1]

    dist = sqrt((x1-x2)**2 + (y1-y2)**2)
    return dist


def convert_bit(j, binary):
    """
    Convert a specific bit in binary strong from 1 to 0
    
    Args:
    j (integer) -- an index of binary to convert
    binary (string) -- represets the binary before conversion
    
    Returns:
    bit (string) -- represets the binary after conversion
    """
    
    bit = ""
    for i in range(0, len(binary)):
        if i == j and binary[i] == "1":
            bit += "0"
        else:
            bit += binary[i]
            
    return bit


def find_minimum(j, A, D, binary, num_cities):
    """
    Find minimum of sum of specific values from dictionary A and 2-dimensional array D
    
    Args:
    j (integer) -- a particular index to convert bit
    A (dictionary) -- stores a binary string as key and an array as value
    D (list of list) -- 2-dimetional array that holds euclidean distances between all coordinates
    binary (string) -- a binary 
    num_cities (integer) -- the number of cities to vist (size n of the problem)
    
    Returns:
    smallest (integer) -- minimun cost of the trip
    """
    
    smallest = 1000000
    for k in range(0, num_cities):
        if k != j and binary[k] == "1":
            new_binary = convert_bit(j, binary)
            candidate = A[new_binary][k] + D[j][k]
            if candidate < smallest:
                smallest = candidate

    return smallest


def travelling_salesman_problem(data_dict, num_cities):
    """
    Implement dynamic programming algorithm to solve travelling salesman problem
    
    Args:
    data_dict (dictionary ) --  storing coordinates 
    num_cities (integer) -- the number of cities in total
    
    Returns:
    smallest (integer) -- minimun cost of the trip
    """
    
    # Construct binary string where 1 means the node presented by that index exists in the set, 0 otherwise
    num_cases_of_set = 2**(num_cities-1)
    S = []
    generate_binary(num_cases_of_set, S, num_cities-1)

    # The set S must contain the fist node, this first digit of binary must always be 1
    del S[len(S)-1]
    zeros = ""
    for i in range(0, num_cities-1):
        zeros += "0"
    S.insert(0, zeros)

    for i in range(0, len(S)):
        S[i] = "1" + S[i]

    # print("S")
    # print(S)

    # Initialize dictionary A
    A = {}
    for binary in S:
        A[binary] = []
        if binary[0] == "1" and count_ones(binary) == 1:
            A[binary].append(0)
        else:
            A[binary].append(10000000)
        for i in range(1, num_cities):
            A[binary].append(10000000)

    # print("A")
    # print(A)

    # Pre-calculate euclidean distances
    D = np.zeros((num_cities, num_cities))
    for i in range(0, num_cities):
        for j in range(0, num_cities):
            D[i][j] = get_euclidean_distance(i, j, data_dict)


    for m in range(2, num_cities+1):
        # for each set in S with m number of 1's
        for binary in find_ones(m, S):
            # for 1's in S that is not the first 1
            for j in range(1, len(binary)):
                if binary[j] == "1":
                    A[binary][j] = find_minimum(j, A, D, binary, num_cities)

    # print("final A")
    # print(A)

    all_in_set = ""
    smallest = 10000000
    for char in range(0, num_cities):
        all_in_set += "1"
    for j in range(0, num_cities):
        candidate = A[all_in_set][j] + get_euclidean_distance(j, 0, data_dict)
        if candidate < smallest:
            smallest = candidate

    return int(np.floor(smallest))

In [ ]:
tuple_obj = open_file("data/traveling-salesman-problem-test1.txt")
assert(travelling_salesman_problem(tuple_obj[0], tuple_obj[1]) == 7)

tuple_obj = open_file("data/traveling-salesman-problem-test2.txt")
assert(travelling_salesman_problem(tuple_obj[0], tuple_obj[1]) == 10)

tuple_obj = open_file("data/traveling-salesman-problem-test3.txt")
assert(travelling_salesman_problem(tuple_obj[0], tuple_obj[1]) == 12)

tuple_obj = open_file("data/traveling-salesman-problem-test4.txt")
assert(travelling_salesman_problem(tuple_obj[0], tuple_obj[1]) == 14)

tuple_obj = open_file("data/traveling-salesman-problem.txt")
assert(travelling_salesman_problem(tuple_obj[0], tuple_obj[1]) == 26442)

## Approximation algorithm for NP complte problem (heuristics)

Greedy algorithm
- ideally items with large value, but small size
- sort adn reindex items so that
    - $\dfrac{v_{1}}{w_{1}} \ge \dfrac{v_{2}}{w_{2}} \ge \dots \ge \dfrac{v_{n}}{w_{n}}$
- pack items in this order until one doesn't fit    
- return either solution from above, or the maximun valuable item, whichever is better

Dynamic programming
- for a user-specified parameter $\epsilon \gt 0$, guarantee a $(1-\epsilon)$ approximation
- running time will increase as $\epsilon$ decreases
- If $w_{i}$'s and $W$ are integers, can solve knapsack problem via dynamic programming in $O(nW)$ time
- If $v_{i}$'s are integers, can solve knapsack problem via dynamic programming in $O(n^{2}v_{max})$ time
1. round each $v_{i}$ down to the nearest multiple of $m$ (where $m$ depends on $\epsilon$)
2. divide the result by $m$ to get $\hat{v_{i}}$'s integers ($\hat{v_{i}}$ = floor($\dfrac{v_{i}}{m}$))
3. solve using dynamic programming with sizes $\hat{v_{1}} \dots \hat{v_{n}}$, weights $w_{1} \dots w_{n}$, and maximun weight limit $W$

Sub-problem
- for i = 0 to n and x = 0 to n, define $S_{i,x}$ = minimum total size needed to achieve value $\ge x$ while using only the first $i$ items (or $+\infty$ if impossible)

Recurrence
- $S_{i,x} = min\left[S_{(i-1),x}, w_{i} + S_{(i-1),(x-v_{i})}\right]$ where $S_{(i-1),(x-v_{i})} = 0$ if $v_{i} \ge x$

Pseudo code
- let $A$ = 2-D array ($i$ = $0$ to $n$, $x$ = $0$ to $nv_{max}$)
- base case: $A[0,x]$ = $0$ if $x = 0$, $+\infty$ otherwise
- for i = 1 to n
    - for x = 0 to $nv_{max}$
        - $A[i,x]$ = $min\left[A[i-1,x], w_{i} + A[i-1,x-v_{i}]\right]$ where $A[i-1,x-v_{i}] = 0$ if $v_{i} \ge x$
- return the largest $x$ such that $A[n,x] \le W$

In [ ]:
from math import sqrt


def open_file(file_path):
    """
    Read-in a file containing rows of data

    Args:
    file_path -- location of file to read

    Returns:
    (data_dict, num_vertices,  num_edges) -- a tuple with a dictionary representing a graph and integers reprsenting number of vertices and edges
    """

    data_dict = {}
    
    with open(file_path, 'r') as line:
        data_array = line.read().split("\n")
        num_cities = int(data_array[0])
        del data_array[0] # delete first element, which is just metadata
        for item in data_array:
            index = int(item.split(" ")[0])
            x = float(item.split(" ")[1])
            y = float(item.split(" ")[2])
            data_dict[index] = (x,y)
            index += 1
    return (data_dict, num_cities)


def find_closest_city(city, data_dict, S):
    """
    Find a neighbouring city that has the minimum distance from the current city
    
    Args:
    city - an integer representing a city
    data_dict - a dictionary holding (x,y) coordinates of each city
    S -- a dictionary to keep all visited cities
    
    Returns:
    (city_with_minimum_dist, minimum_dist) -- tuple of an integer (city with minimum distance) and an integer (minimum distance)
    """
    
    city_with_minimum_dist = 0
    minimum_dist = 10000000
    
    x1 = data_dict[city][0]
    y1 = data_dict[city][1]
    
    for key in data_dict:
        if key not in S:
            x2 = data_dict[key][0]
            y2 = data_dict[key][1]

            dist = sqrt((x1-x2)**2 + (y1-y2)**2)
            if dist < minimum_dist:
                minimum_dist = dist
                city_with_minimum_dist = key
            elif dist == minimum_dist:
                if city_with_minimum_dist > key:
                    city_with_minimum_dist = key
        
    return (city_with_minimum_dist, minimum_dist)


tuple_obj = open_file("data/traveling-salesman-problem-heuristic.txt")
# tuple_obj = open_file("data/traveling-salesman-problem-heuristic-test1.txt")
# tuple_obj = open_file("data/traveling-salesman-problem-heuristic-test2.txt")
# tuple_obj = open_file("data/traveling-salesman-problem-heuristic-test3.txt")
data_dict = tuple_obj[0]
num_cities = tuple_obj[1]
# print(num_cities)

S = {}
D = []
S[1] = "visited" # start tour at the first city
num_visited_cities = 1
current_city = 1


while num_visited_cities < num_cities:
    result = find_closest_city(current_city, data_dict, S)
    S[result[0]] = "visited"
    current_city = result[0]
    D.append(result[1])
    num_visited_cities += 1
    
    
x1 = data_dict[current_city][0]
y1 = data_dict[current_city][1]
x2 = data_dict[1][0]
y2 = data_dict[1][1]
final_dist = sqrt((x1-x2)**2 + (y1-y2)**2)
print(sum(D)+final_dist)
# 15.2361
# 23
# 716
# 1203406.5012708856

## Maximum cut problem

- given $G = (V,E)$, a cut $(A,B)$ that maximizes the number of crossing edges
- for a cut $(A,B)$ and a vertex $V$, define
    - $c_{v}(A,B)$ = number of edges incident on $v$ that corss $(A,B)$
    - $d_{v}(A,B)$ = number of edges incident on $v$ that don't corss $(A,B)$
    
## Local search algorithm

- let $(A,B)$ be an arbitrary cut of $G$
- while there is a vertex $v$ with $d_{v}(A,B) \gt c_{v}(A,B)$
    - move $v$ to other side of the cut (increase number of crossing edges by $d_{v}(A,B) - c_{v}(A,B) \gt 0$)
- return final cut $(A,B)$

## Neighbourhoods

- let $x$ = set of candidates solutions to a problem for each $x$ in $X$, specify which $y$ in $X$ are its neighbours
- $x,y$ are neighbouring cuts iff differ by moving one vertex
- $x,y$ are neighbouring variable assignments iff differ in the value of a single variable
- $x,y$ are neighbouring TSP tours iff differ by 2 edges

## Generic local search algorithm

- let $x$ = some initial solution
- while the current solution $x$ has a superior neighbouring solution $y$: set $x = y$
- return the final (locally optimal) solution $x$

FAQ
- how to pick initial soltion $x$? random / best heuristic
- if there are several superior neighbouring $y$, which to choose? random / biggest improvement
- how to define neighbourhoods? find "sweat spots" between solution quality and efficient searchability
- is local search guaranteed to terminate? if $x$ is finite and every local step improves some objective function, then yes
- is local search guaranteed to converge quickly? usually not
- are locally optimal solutions generally good approximations to globally optimal ones? no (but you can run many times and pick the best)

## 2-SAT

- given $n$ Boolean variables $x_{1} \dots x_{n}$ (True or False) and $m$ clauses of 2 literal each ($x_{i}$ or $!x_{i}$)
- return "yes" if there is an assignment that simultaneously satisfies every class. "no" otherwise
- Ex. "yes" when $x_{1} = x_{3}$ = TRUE and $x_{2} = x_{4}$ = FALSE 

Can be solved in polynomial time!
- reduction to computing strongly connected components
- "backtracking" works in polynomial time
- randomized local search

## 3-SAT

- NP complete

## Papadimitriou's 2-SAT algorithm

Repeat $log_{2}n$ times (n = number of variables)
- choose random initial assignment
- repeat $2n^{2}$ times
    - if current assignment satisfies all clauses, halt and report this
    - else, pick arbitrary unsatisfied clause and flip the value of one of its variables (choose between the two unformly at random)
- report "unsatisfiable"

Advantages
- runs in polynomial time
- always correct on unsatisfiable instances

## Random walks

- at each time step, your position goes up or down by 1, with 50/50 chance (except if at position 0, in which case you move to position 1 with 100 chance)
- for an integer $n \ge 0$, let $T_{n}$ = number of steps until random walk reaches position n
- $E[T_{n}] = n^2$ (corollary: $Pr[T_{n} > 2n^{2}] \ge \dfrac{1}{2}$)

In [ ]:
def open_graph(file_path):
    """
    Imports a file and stored data into a dictionary

    Args:
    file_path -- location of file

    Returns:
    graph -- dictionary representation of graph
    """

    graph = {}
    rep = {}
    num_nodex = 0

    with open(file_path, 'r') as line:
        array = line.read().split("\n")
        num_nodes = int(array[0]) * 2
        del array[0]
        
        for i in range(-num_nodes, num_nodes+1):
            if i != 0:
                graph[i] = []
        
        for subarray in array:
            x1 = int(subarray.split(" ")[0])
            x2 = int(subarray.split(" ")[1])

            if x1 > 0 and x2 > 0:
                graph[-x1].append(x2)
                graph[-x2].append(x1)
                rep[-x1] = "exist"
                rep[-x2] = "exist"
            elif x1 < 0 and x2 > 0:
                graph[x1].append(x2)
                graph[-x2].append(-x1)
                rep[x1] = "exist"
                rep[-x2] = "exist"
            elif x1 > 0 and x2 < 0:
                graph[-x1].append(-x2)
                graph[x2].append(x1)
                rep[-x1] = "exist"
                rep[x2] = "exist"
            elif x1 < 0 and x2 < 0:
                graph[x1].append(-x2)
                graph[x2].append(-x1)
                rep[x1] = "exist"
                rep[x2] = "exist"

    return (graph, rep, num_nodes)

        
tuple_obj = open_graph("data/2sat1.txt")    
graph = tuple_obj[0]
rep = tuple_obj[1]
num = tuple_obj[2]
# graph_list = [[1,4],[2,8],[3,6],[4,7],[5,2],[6,9],[7,1],[8,5],[8,6],[9,7],[9,3]]
# num_nodes = 9
# print(graph)
print(num_nodes)

# Remove vertices that don't point to any other vertices
keys_to_delete = []
for key in graph:
    if graph[key] == []:
        keys_to_delete.append(key)
        
for key in keys_to_delete:
    del graph[key]
        
# print(graph)

# Apply reduction where there is only 1 representation (not 2) of a variable
values_to_delete = []
for i in range(-num_nodes, num_nodes+1):
    if i != 0:
        if i in rep and -i in rep:
            print(i + "is good")
        else:
            values_to_delete.append(i)
                        
for i in values_to_delete:
    if i in graph:
        del graph[i]
            
for i in values_to_delete:    
    for key in graph:
        if i in graph[key]:
            graph[key].remove(i)
      
# print(graph)
# 0
# 101100